# COM-402 Password Cracking and PAKE Implementation Homework Solution

Password cracking is kind of the _Hollywood_ way of hacking. But contrary to what Horacio taught us, this is not always suitable. For example, against a web login, this is a very bad idea, as each try is monitored, takes a couple of seconds, and may result in you being blocked.

Though, there are certain situations when this is doable. In particular, when you have at hand encrypted passwords. But why would they be encrypted? And how?

When you deal with users and passwords, it's very _very_ bad practice to store plaintext passwords. What you do is store the password as a hash and when the user logs in, you compute the hash of its input and compare it to the stored hash. You can also add some subtleties, such as salt or time-dependent hashes. But the idea is always the same.

Now, suppose your database gets hacked and dumped. Everyone on the internet has access to your database. Now, hackers have a list of hashes, that match usernames. This is, in itself, useless as the hashes are non-reversible and can't be used to log in. That's when password cracking shines: you have at hand a bunch of hashes, and your goal is to reverse them to find the original passwords.

**Note**:
For all password cracking, it's always a good idea not to try and find one hash, but to compute a hash and compare it to all the unsolved hashes. This will save a lot of time, rather than doing the whole process multiple times.

In [ ]:
import hashlib
import itertools
import multiprocessing
import time

## Part 1: Brute force attack
### Understanding the task
Bruteforce attacks are the simplest form of password cracking. Basically, you try all the combinations one after the other. Given that you do it well, this method is guaranteed to reverse the hash... some day. Because the complexity and thus length of the task is directly proportional to the length of the password, and the size of the character set. More precisely: say you try all combinations from a set of 36 characters (lowercase letters and all numbers), with a password length of 5, you'll have to try $36^5 = 60466176$ different passwords.

For example, a hacker knowing that all passwords consist of fewer than 5 characters with only lowercase letters and numbers will laugh. But having upper and lower cases, numbers, some special characters and a 16-length password, you're in for about $10^{29}$. Practically unbreakable.

### The methods
There are several methods for you to try and hack the hashes. In a real scenario, you'd want to rely on existing software. The most well-known and used are [John the Ripper](https://www.openwall.com/john/) and [Hashcat](https://hashcat.net/hashcat/). They support many hash types, are optimized to run as fast as possible and can even leverage your GPU(s).

For the sake of comprehension, here we will use a custom python script. Python is great, but not necessarily in this situation, because it's a quite heavier language, compared to say C or C++. Nonetheless, this is far simpler to demo on that.

### Code

In [ ]:
end = False
def block_func(combination):
    """Take one combination, compute its hash, and return if a match is found"""
    if end: 
        return
    password = "".join(combination)
    h = hashlib.sha256(password.encode())
    digest = h.hexdigest()
    if digest in all_hashes:
        print("{} === {}".format(digest, password))
        return digest

# all the possible characters
charset = "abcdefghijklmnopqrstuvwxyz1234567890"

# the list of all the hashes
all_hashes = set([
    "19dbaf86488ec08ba7a824b33571ce427e318d14fc84d3d764bd21ecb29c34ca",
    "dd9ad1f17965325e4e5de2656152e8a5fce92b1c175947b485833cde0c824d64",
    "845e7c74bc1b5532fe05a1e682b9781e273498af73f401a099d324fa99121c99",
    "a6fb7de5b5e11b29bc232c5b5cd3044ca4b70f2cf421dc02b5798a7f68fc0523",
    "1035f3e1491315d6eaf53f7e9fecf3b81e00139df2720ae361868c609815039c"
    ])
hashes_to_crack = len(all_hashes)


begin = time.time()

# create a pool of processes
pool = multiprocessing.Pool(processes=1)
total_matches = []
for length in range(4, 6):
    # create the set of passwords for this length
    combinations_generator = itertools.product(charset, repeat=length)

    # compute number of possible passwords
    total_combinations = len(charset)**length

    # read the generator lazily and map combinations to the function
    for i, x in enumerate(pool.imap_unordered(block_func, combinations_generator, 1000)):
        if x is not None:
            total_matches.append(x)
            if len(all_hashes) == len(total_matches):
                break
        if i % 100 == 0:
            # display your progress
            print("Progress for length {}: {:.3f}%".format(length,100*i/total_combinations), end="\r")

end = time.time()
print("\nTime for naïve: {:.3f}s".format(end-begin))

This code runs within a few hours on a laptop CPU. Not great, not terrible.

We obtain the following corresponding passwords:
```
19dbaf... === szpn9
dd9ad1... === gi02n
845e7c... === j67c
a6fb7d... === bgfvf
1035f3... === 2vdxm
```
As you probably realize, this kind of attack can't be reasonably used at a large scale. This is the last resort method, that can't be deployed for large passwords, or too complicated passwords.

### Leveraging your multi-core machine
Though, this task is embarrassingly parallel. If you had a thousand threads, you can easily split your task among them, and your computation would be roughly a thousand times faster. Good libraries such as Hashcat or John actually do that, and can use your CPU at its maximum capacity. This is also how GPUs are leveraged: they contain thousands of "weak cores". Individually they are not great for heavy computing, but for such task they are perfect.

Here is an example of a parallelized code that uses your CPU resources a bit better:

In [ ]:
end = False
def block_func(combination):
    """Take one combination, compute its hash, and return if a match is found"""
    if end:
        return
    password = "".join(combination)
    h = hashlib.sha256(password.encode())
    digest = h.hexdigest()
    if digest in all_hashes:
        print("{} === {}".format(digest, password))
        return digest

# all the possible characters
charset = "abcdefghijklmnopqrstuvwxyz1234567890"

# the list of all the hashes
all_hashes = set([
    "19dbaf86488ec08ba7a824b33571ce427e318d14fc84d3d764bd21ecb29c34ca",
    "dd9ad1f17965325e4e5de2656152e8a5fce92b1c175947b485833cde0c824d64",
    "845e7c74bc1b5532fe05a1e682b9781e273498af73f401a099d324fa99121c99",
    "a6fb7de5b5e11b29bc232c5b5cd3044ca4b70f2cf421dc02b5798a7f68fc0523",
    "1035f3e1491315d6eaf53f7e9fecf3b81e00139df2720ae361868c609815039c"
    ])
hashes_to_crack = len(all_hashes)


begin = time.time()

# create a pool of processes
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
total_matches = []
for length in range(4, 6):
    # create the set of passwords for this length
    combinations_generator = itertools.product(charset, repeat=length)

    # compute number of possible passwords
    total_combinations = len(charset)**length

    # read the generator lazily and map combinations to the function
    for i, x in enumerate(pool.imap_unordered(block_func, combinations_generator, 1000)):
        if x is not None:
            total_matches.append(x)
            if len(all_hashes) == len(total_matches):
                break
        if i % 100 == 0:
            # display your progress
            print("Progress for length {}: {:.3f}%".format(length,100*i/total_combinations), end="\r")

end = time.time()
print("\nTime for naïve: {:.3f}s".format(end-begin))

### Code explanation
The above code is quite simple, but not perfect. It uses `imap_unordered`, which will do the following:
- Read the generator lazily by chunks of 1000 (you don't want to have _all_ the combinations at once in memory);
- Then the 16 workers generated earlier will pick the combinations one after the other and apply each combination to the defined function;
- Because it's unordered, if a worker finishes early, he doesn't have to wait until his predecessor is done, because the output can be unordered.

This leverages the computing power of your CPU better, but is still not perfect. The main defect is that it creates a lot of computing overhead. Each "block" (a combination) is quite fast, so a worker is often starting and finishing. It would be better to give to each worker a chunk of combinations, and they each spend some time iterating within them. But for now, good enough.


## Part 2: Dictionary attack with rules
As you saw in the previous part, exhaustive attacks while very powerful are not a viable solution for many passwords. But (un)fortunately, humans are not random machines, and strongly dislike passwords of the type `dn8S89?/8+F\f'hk7|è`, while they can simply remember `password123`. Even in 2021, those are still [awfully used](https://en.wikipedia.org/wiki/List_of_the_most_common_passwords#SplashData).

### How it works
The idea behind dictionary attacks is just that: humans are more likely to generate their passwords using words from a dictionary. So if you have your hand on a dictionary, you can try all the words in it. For all intents and purposes, a dictionary is a list of unique words, likely to be used by humans.

Let's start with a simple dictionary. Take the Oxford dictionary, it contains about 171k words. Not too bad, finding all the hashes may take a couple of seconds on a laptop CPU. You may find a few passwords with that.

But what if someone uses `Password`? Well, that's the next step. Now that you have your list of words, it's better to try some tweaks. Try some capital letters, change letters (`o -> O -> 0`, `A -> 4`). The next step is to combine words. Concatenate 2,3,4 or 10 words, try combinations.

And what about `motdepasse`? Yet another step. Your dictionary may not be exhaustive, and you only cover English speakers. Well, download a French, German, or Finnish dictionary, and add them to your initial one.

And what about using your mother's name, Martha, in the password? Yes, possible. Retrieve a list of human names. And also city names. And celebrities. And events. And slurs. And everything. Oh, and don't forget non-alphanumerical characters (`!,ç%&+"...`)

**It's exploding!!**

By doing all these steps, you realize you now have a database of several billion or more words to try. Are you really in a better spot compared with brute force? Actually yes. Because now you have the same space size (say, $10^{12}$ passwords), but with brute force that only covered the passwords less than 5 characters, on a small character set. Now your huge dictionary covers `P4ssw0rd123!` which is more likely to be used rather than `4D/x.`. With brute force, you'd never reach that password.

Hackers spend years enhancing their dictionaries. If you're smart, you also sort these words in a manner that allows smart selection. For example, you're hacking a German music platform. Chances are, your Finnish and fishing-related dictionaries are not useful there. If you target a specific person or entity, you also must have a "personal" dictionary (their pet's name, date of birth, significant other's name,...).

Having a good dictionary is tedious, but may really help you when cracking passwords.

### Parallelism
As before, this task is embarrassingly parallel. Each entry in your dictionary is independent of the others (this is not exactly true, but good enough for us). Thus, to gain a lot of time, you can re-use some code from before and leverage your cores.

### Commutativity
On your first try, you may try the following: take all your modifications, and create all the combinations (4 modifications, that gives 16 different outcomes, including the "no-modify" one). While this is absolutely OK, there may be one or more passwords that will remain unbreakable. The reason for that is commutativity. The first modification possible (change capital letters) will be affected by the other transformations.

Take `window`, the example given. If you first _title-ize_ it, then replace the numbers, you'll get `W1nd0w`. If you first replace the `i`s, then _title-ize_, then replace the `o`s, you'll get `W1Nd0w`. And first replacing the numbers, then _title-ize_, you obtain `W1Nd0W`. These three will yield different hashes.

It is thus important to take all combinations, with respect to the order. That gives 65 possible combinations of these modifications (again, including the empty modification). Don't worry though, if you code it smartly, many combinations are commutative, and this rarely matters. After filtering, you'll have to compute only a handful of versions of the password, usually between 1 and 20.

### The code (finally)
#### Generating the combinations
One good way to generate all combinations of modifications is to take advantage of the fact that Python deals with functions as objects. This allows to create an array of the unique modifications, and then create combinations from that. Here, we define named functions for the sake of completeness, but as the functions are almost trivial, lambda functions could have done the trick.

Also, we use [hints](https://docs.python.org/3/library/typing.html), available from python 3.5, that make the development a lot easier.

```python
def modif1(p: str):
    return p.title()

def modif2(p: str):
    return p.replace("e", "3")

def modif3(p: str):
    return p.replace("o", "0")

def modif4(p: str):
    return p.replace("i", "1")

all_modifs_combinations = set()
all_modifs = [modif1, modif2, modif3, modif4]
for length in range(1, len(all_modifs)+1):
    for comb in itertools.permutations(all_modifs, length):
        all_modifs_combinations.add(comb)
```

#### Creating the combinations
```python
# let p be a given entry from the dictionary
all_versions = set([p])
for comb in all_modifs_combinations: # go through each set of modifications
    p_temp = p
    for modificator in comb: # apply them fo the base password iteratively
        p_temp = modificator(p_temp)
    all_versions.add(p_temp) # all_versions is a set, so no duplicate will remain
```

#### Hashing
```python
for version in all_versions:
    hash = hashlib.sha256(version.encode()).hexdigest()
    if hash in all_hashes:
        print("{} === {} (from {})".format(hash, version, p))
        all_hashes.remove(hash)
        return p, hash
```
#### Complete, with parallelism

In [ ]:
all_hashes = set(["3d43987eeb0e001390791134ea47511a9758eaba9e07d61bcfae76323cdc9d14",
                  "824aea9643be485d330860886e41599e26e190dd4c6eee203c80f1247ea5457b",
                  "002ff8b4fb4538e0a44a374e45898e7140e24ef2be7814ccd71eafce946db60e"])
def modif1(p: str):
    return p.title()
def modif2(p: str):
    return p.replace("e", "3")
def modif3(p: str):
    return p.replace("o", "0")
def modif4(p: str):
    return p.replace("i", "1")

def modif_and_hash(p):
    """Try all modifications and hashes from one given base password."""
    if len(all_hashes) == 0:
        return
    p = p.replace("\n", "") # ensure there are no \n at the end
    # create the alternative from the base passwords
    all_versions = set([p]) # include the base password
    for comb in all_modifs_combinations:
        p_temp = p
        for modificator in comb:
            p_temp = modificator(p_temp)
        all_versions.add(p_temp)
    # Hash and compare each of them
    for version in all_versions:
        hash = hashlib.sha256(version.encode()).hexdigest()
        if hash in all_hashes:
            print("{} === {} (from {})".format(hash, version, p))
            all_hashes.remove(hash)
            return p, hash

# generating all possible combinations of password modifications
all_modifs_combinations = set()
all_modifs = [modif1, modif2, modif3, modif4]
for length in range(1, len(all_modifs)+1):
    for comb in itertools.permutations(all_modifs, length):
        all_modifs_combinations.add(comb)

file = open("rockyou.txt", encoding="latin-1")
pool = multiprocessing.Pool(multiprocessing.cpu_count()) # define a pool of workers
results = []

# read the file by chunks of 10k rows at a time, then feed one
# rows one after the other to a worker
for r in pool.imap_unordered(modif_and_hash, file, 10000):
    if r is not None:
        print(r)
        results.append(r)

file.close()

### Passwords
- `3d43987eeb0e001390791134ea47511a9758eaba9e07d61bcfae76323cdc9d14 === Myd0Gg010`
- `824aea9643be485d330860886e41599e26e190dd4c6eee203c80f1247ea5457b === s1llycat19`
- `002ff8b4fb4538e0a44a374e45898e7140e24ef2be7814ccd71eafce946db60e === j0hnh3ad`

### Final thoughts
As you can observe, this method allows to test not more or less, but different passwords. If you were absolutely positive the password was random (e.g. generated by a password manager), then this method is useless. But against passwords of most people, this is a very powerful attack. No wonder hackers take years to complete their own dictionaries! Also, just as before, it would be possible to generate in advance a list of all hashes, and create a lookup table (or a [rainbow table](https://lasec.epfl.ch/pub/lasec/doc/Oech03.pdf), courtesy of Philippe Oeschlin, teacher of fall 2020). Next exercise will show you one protection against that.

## Part 3: Hashes get salty
### Why use salt?
The previous attack, while powerful in theory, is often countered. If you push forward the reasoning there once you have your list of words, you can just compute a lookup table of all the hashes of all those passwords, and use it. You just have to do the work once, and then de-hashing is $\mathcal{O}(1)$! Good for us, no good for others. But if every password you store has a "unique" salt at the end, this "do the work once" must be done once per salt. Supposing, as mentioned, a 2 hexadecimal characters salt, this increases your work by a factor 256! The good thing is that you don't even need to keep the salt secret (well, it's always better). If the password is salted, chances are there is no lookup table for those hashes.

### WPA2 Trivia
With WPA2 Personal, [routers salt the password with the SSID of the network](https://crypto.stackexchange.com/questions/28975/encryption-algorithm-used-in-wpa-wpa2). But some big routers always came with the same default SSID that people never bothered changing (_linksys_, _asus_, _orange_,...). Some are so common that people started [collecting them](https://wigle.net/stats#ssidstats). Once you have the most common, you can start creating lookup tables and rainbow tables, specific for each of these common SSIDs. For this reason, amongst others, modern routers now come with a default SSID appended by some pseudo-random value (like _linksys-SJF3O-F82ND-F8F3K_).

### How to attack
The idea is very similar to the previous exercise. Now, instead of applying some functions to your password, you just try to append each salt and try it. Because we provide you with all the salts used, you only have to generate 10 versions for each password you consider, and hash each of them. But if we hadn't, or if the salt was partially dynamic, we would need to try all the salts for all the passwords. For example, [passlib](https://passlib.readthedocs.io/en/stable/index.html) (a standard authentication library for python) lets you specify a 0-16 alphanumerical characters salt or [will generate one at random each time](https://passlib.readthedocs.io/en/stable/lib/passlib.hash.sha256_crypt.html#passlib.hash.sha256_crypt). With such method, you can't rely on rainbow tables at large scales.

### The code
We mostly reuse the code from before:

In [ ]:
all_hashes = set(["69839ca8ae00839a988b8e0260091d15425df1265becd4548763008284a2ea50",
                  "ce85f06a4ddf38633149a9098cdd1d672f61c109a4348ca116441ddaa2129b9e",
                  "858930e3415097b129fdd54dda2032fffb378a4d07eaf1fce69158890bb6a242"])

def salt_and_hash(p):
    """Take one password, and hash it using all the possible salts."""
    salts = ["b9", "be", "72"]
    p = p.replace("\n", "") # remove possible trailing \n
    for s in salts:
        salted = p+s
        hash = hashlib.sha256(salted.encode()).hexdigest()
        if hash in all_hashes:
            print("{} === {} (salt {})".format(hash, p, s))
            return p, hash

file = open("rockyou.txt", encoding="latin-1")
pool = multiprocessing.Pool(multiprocessing.cpu_count()) # define a pool of workers
results = []

# read the file by chunks of 10k rows at a time, then feed one
# rows one after the other to a worker
for r in pool.imap_unordered(salt_and_hash, file, 10000):
    if r is not None:
        print(r)
        results.append(r)

file.close()

### Passwords
* `69839ca8ae00839a988b8e0260091d15425df1265becd4548763008284a2ea50 === amutcat` (salt `b9`)
* `ce85f06a4ddf38633149a9098cdd1d672f61c109a4348ca116441ddaa2129b9e === HUMMA` (salt `be`)
* `858930e3415097b129fdd54dda2032fffb378a4d07eaf1fce69158890bb6a242 === i_hat_you_a` (salt `72`)

### Closing thoughts
As you saw, this is quite straightforward. But remember that in the wild, you may not know the list of potential salts, and may have to explore a huge list _for each password_. If you were to know that the hashes were always 2 hexa characters, this is 256 times harder than a normal dictionary attack.


## Part 4: A CTF challenge

The source of this challenge is [here](https://ctf-wiki.org/en/crypto/hash/attack/).In the challenge, the hash algorithm is quite complex: multiple hash algorithms are used, the password is randomly salted, the total length of the password and salt is long enough, and the hashing iterates for 32 rounds. These are all good practices to make the password cracking harder. It takes a lot of time to crack the password using the methods like brute force or rainbow table attacks.

However, not combining all these methods carefully can make all those good practices in vain.

The given hash algorithm is as follows:

for i in range(0, 32):
$$
salt_{i+1} = salt_{i} \oplus hash\_{round}_{[31-i]}(hash_{i}) \\
hash_{i+1} = hash_{i} \oplus hash\_{round}_{[i]}(salt_{i+1})
$$

With a simple xor transformation, we can get:

for i in range(0, 32):
$$
salt_{i} = salt_{i+1} \oplus hash\_{round}_{[31-i]}(hash_{i}) \\
hash_{i} = hash_{i+1} \oplus hash\_{round}_{[i]}(salt_{i+1})
$$

This means that we can reverse the hash algorithm by iterating from the last round to the first round.

Specifically, we know $hash_{32}$, $salt_{32}$, and $hash\_round_{[31]}$, so when $i = 31$, we can get $hash_{31}$ and $salt_{31}$ using the following equations:
$$
hash_{31} = hash_{32} \oplus hash\_{round}_{[31]}(salt_{32}) \\
salt_{31} = salt_{32} \oplus hash\_{round}_{[0]}(hash_{31})
$$

Then we can get $hash_{30}$ and $salt_{30}$, $hash_{29}$ and $salt_{29}$, ..., until we get $hash_0$ and $salt_0$.

Therefore, we can crack the password and salt at round 0 as follows:

for i in range(31, -1, -1):
$$
hash_{i} = hash_{i+1} \oplus hash\_{round}_{[i]}(salt_{i+1}) \\
salt_{i} = salt_{i+1} \oplus hash\_{round}_{[31-i]}(hash_{i})
$$

In summary, the key to solving this challenge is to reverse the hash algorithm. The solution script is as attached as ext4_sol.py.